# Item:
- propteries: [id, name, price]
- methods: getter + setter, toString, compareTo

In [6]:
class Item:
    def __init__(self, id: int, name: str, price: float):
        self.__id = id
        self.__name = name
        self.__price = price

    @property
    def id(self) -> int:
        return self.__id

    @property
    def name(self) -> str:
        return self.__name

    @property
    def price(self) -> float:
        return self.__price
    
    @id.setter
    def id(self, id: int):
        self.__id = id
  
    @name.setter
    def name(self, name: str):
        self.__name = name

    @price.setter
    def price(self, price: float):
        self.__price = price  
        
    def __str__ (self):
        return f"Item(id={self.__id}, name='{self.__name}', price={self.__price})" 
    
    # so sanh 2 item 
    def __lt__ (self, other):
        # kiem tra other la thuc the cua Item
        if isinstance(other, Item):
            return self.__id < other.__id
        return NotImplemented
    
    def compare_to(self, other):
        if not isinstance(other, Item):
            raise ValueError("Can only compare with another Item!")
        return self.__id - other.__id

In [11]:
item_catalog = {
    1: Item(1, "Milk", 2.49),
    2: Item(2, "Bread", 1.99),
    3: Item(3, "Eggs", 3.25),
    4: Item(4, "Apples", 4.50),
    5: Item(5, "Bananas", 2.99),
    6: Item(6, "Cheese", 5.75),
    7: Item(7, "Tomatoes", 3.10),
    8: Item(8, "Chicken", 8.99),
    9: Item(9, "Rice", 6.20),
    10: Item(10, "Coffee", 7.80),
}

# Account: 
- properties: [id, name, score]
- methods: getLoyaltyPromo() - 15%

In [28]:
class Account:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name
        self.score = 0

    def getLoyaltyPromo(self):
        """15% discount if score is high (>= 100), else no discount"""
        return 0.15 if self.score >= 100 else 0.0

    def __str__(self):
        return f"Account(id={self.id}, name='{self.name}', score={self.score})"


# Order:
- properties: [id, account, item_list (dict), voucher (str)]
- methods: getVoucherPromo() - 10%, getBulkOrderPromo() - 10%, setScore(), printBill(), total()

In [29]:
class Order:
    def __init__(
        self, id: str, account: Account, item_list: dict = {}, voucher: str = ""
    ):
        self.__id = id
        self.account = account
        self.__item_list = item_list
        self.__voucher = voucher

    @property
    def id(self):
        return self.__id

    @property
    def item_list(self):
        return self.__item_list

    @property
    def voucher(self):
        return self.__voucher

    @item_list.setter
    def item_list(self, item_list: dict):
        self.__item_list = item_list

    @voucher.setter
    def voucher(self, voucher: str):
        self.__voucher = voucher

    # add item
    def add_item(self, item: Item):
        if item.id not in self.__item_list:
            # them moi
            self.__item_list[item.id] = 1
        else:
            # chinh danh sach + dieu chinh qty
            self.__item_list[item.id] += 1

    # remove item
    def remove_item(self, item: Item):
        if item.id in self.__item_list:
            # tru 1 qty
            self.__item_list[item.id] -= 1
            if self.__item_list[item.id] == 0:
                # xoa item
                del self.__item_list[item.id]
                return True
            return True
        return False

    # promo -----------
    def getVoucherPromo(self):
        return 0.10 if self.voucher else 0.0

    def __getTotalQuantity(self):
        return sum(self.__item_list.values())

    def getBulkOrderPromo(self):
        return 0.10 if self.__getTotalQuantity() > 10 else 0.0

    def total(self):
        sum = 0
        for item_id, quantity in self.item_list.items():
            item = item_catalog[item_id]
            line_total = item.price * quantity
            sum += line_total
        # tinh them giam gia
        sum *= (
            1
            - self.getBulkOrderPromo()
            - self.getVoucherPromo()
            - self.account.getLoyaltyPromo()
        )
        return sum

    def setScore(self):
        self.account.score += int(self.total() * 0.1)

    # IN BILL RA FILE TXT
    def printBill(self):
        total_cost = self.total()  # calculate once
        self.setScore()  # update the score before printing

        with open(f"bill_{self.__id}.txt", "w") as file:
            file.write(f"Bill ID: {self.__id}\n")
            file.write(f"Voucher: {self.__voucher}\n")
            file.write(
                f"Loyalty Discount: {self.account.getLoyaltyPromo() * 100}%\n"
            )
            file.write(f"Bulk Order Discount: {self.getBulkOrderPromo() * 100}%\n")
            file.write(f"Total (after discounts): ${total_cost}\n")
            file.write(f"Score (after order): {self.account.score}\n")
            file.write("\nItems:\n")
            for item_id, quantity in self.item_list.items():
                item = item_catalog[item_id]
                line_total = item.price * quantity
                file.write(
                    f"- {item.name} (x{quantity}) [${item.price}] = ${line_total}\n"
                )

In [30]:
acc1 = Account(1, "John")
ord1 = Order("001", acc1, {}, "ABC")
ord1.add_item(item_catalog[1])
ord1.add_item(item_catalog[10])
ord1.add_item(item_catalog[1])
print(ord1.item_list)
ord1.printBill()

{1: 2, 10: 1}
